In [1]:
!nvidia-smi

Sat May  8 06:29:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Model Training** 

In [2]:
import os
import numpy as np
import pickle
import h5py
#import pandas as pd

from __future__ import print_function
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D



In [3]:
classes = 100
current_path = os.path.join(os.getcwd(), 'current_model')
#print(current_path)
#model_name = 'cifar100.h5'
(x_train, y_train) , (x_test, y_test) = cifar100.load_data()
print('x_train_dims : ' , x_train.shape)
print('x_test_dims : ', x_test.shape)
print('y_train_dims : ', y_train.shape)
print('y_test_dims : ', y_test.shape)

print( 'number of training examples available : ', x_train.shape[0])
print('number of testing examples available : ', x_test.shape[0])

y_train = keras.utils.to_categorical(y_train, classes)
y_test = keras.utils.to_categorical(y_test, classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.0
x_test /= 255.0

169009152/169001437 [==============================] - 2s 0us/step
x_train_dims :  (50000, 32, 32, 3)
x_test_dims :  (10000, 32, 32, 3)
y_train_dims :  (50000, 1)
y_test_dims :  (10000, 1)
number of training examples available :  50000
number of testing examples available :  10000


In [4]:
model = Sequential()
# layer one
model.add(Conv2D(128,(3,3), padding = 'same', input_shape = x_train.shape[1:]))
model.add(Activation('elu'))
model.add(Conv2D(128, (3,3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#layer two
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#layer three
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(classes))
model.add(Activation("softmax"))


opt = keras.optimizers.RMSprop(lr=0.0001, decay = 1e-6)

model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])




In [5]:
epochs = 200
data_augmentation = False
predictions = 20
batch_size = 64
validation = []

model.fit(x_train, y_train, batch_size= batch_size, epochs = epochs, validation_data = (x_test, y_test), shuffle = True)
model.save("final_model.h5")
    
        
pickle.dump(validation, open("loss_validation.p",'wb'))

Epoch 1/200
782/782 [==============================] - 63s 38ms/step - loss: 4.2053 - accuracy: 0.0665 - val_loss: 3.6213 - val_accuracy: 0.1556
Epoch 2/200
782/782 [==============================] - 29s 38ms/step - loss: 3.4407 - accuracy: 0.1906 - val_loss: 2.9697 - val_accuracy: 0.2822
Epoch 3/200
782/782 [==============================] - 30s 38ms/step - loss: 3.0224 - accuracy: 0.2650 - val_loss: 2.7236 - val_accuracy: 0.3317
Epoch 4/200
782/782 [==============================] - 30s 38ms/step - loss: 2.7728 - accuracy: 0.3134 - val_loss: 2.5402 - val_accuracy: 0.3625
Epoch 5/200
782/782 [==============================] - 30s 38ms/step - loss: 2.5547 - accuracy: 0.3576 - val_loss: 2.3512 - val_accuracy: 0.4067
Epoch 6/200
782/782 [==============================] - 30s 38ms/step - loss: 2.3952 - accuracy: 0.3932 - val_loss: 2.2216 - val_accuracy: 0.4344
Epoch 7/200
782/782 [==============================] - 30s 38ms/step - loss: 2.2385 - accuracy: 0.4302 - val_loss: 2.1692 - val_ac

# **Model Testing**

In [9]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(32, 32))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 32, 32, 3)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/sample_image.jpeg')
	# load model
	model = load_model('final_model.h5')
	# predict the class
	result = model.predict_classes(img)
	print("The image is of ",result[0])

# entry point, run the example
run_example()

The image is of  42


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
